### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)


# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.


In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

# Add Java Client POM Dependency

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

# Add required Imports

In [4]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.RecordExistsAction;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


# Accessing a record on the Aerospike Server
We build the Key object in Java
We need the namespace and the record digest to find the record on the server.
The record digest is computed by the client library using application provided key (integer, string or byte-array) and the set name. If not in a set, use null for set name.

<img src="../graphics/RecordKey.png"
     alt="Record Digest"
     style="float: left; margin-right: 10px;"
     width="600"
     height="400"/>

In [5]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");
Key key = new Key("test", "testset", "key1");
WritePolicy wP = new WritePolicy();
wP.sendKey = true;
wP.expiration = 300;
wP.recordExistsAction = RecordExistsAction.CREATE_ONLY;
Bin bin1 = new Bin("age", 40);
try {
  client.put(wP, key, bin1);
} catch(Exception e) {
    System.out.printf(e.toString());
}
 
Record record = client.get(null, key);
System.out.print("\nRecord:");
System.out.println(record);

System.out.println("\nRe-insert the record:");
//This should fail for CREATE_ONLY.  We are updating again within ~milliseconds.
try {
  client.put(wP, key, bin1);
} catch(Exception e) {
    System.out.println(e.toString());
}

Record record = client.get(null, key);
System.out.print("\nRecord:");
System.out.println(record);

Initialized the client and connected to the cluster.

Record:(gen:1),(exp:453579683),(bins:(age:40))

Re-insert the record:
com.aerospike.client.AerospikeException: Error 5,1,0,30000,1000,0,A1 127.0.0.1 3000: Key already exists

Record:(gen:1),(exp:453579683),(bins:(age:40))


In [6]:
%sh asadm --enable -e "manage sindex create numeric idx_age ns test set testset bin age"

In [7]:
// get() will not return user stored key in Record object. However, an SI query will via RecordSet.
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.policy.QueryPolicy;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,50));
QueryPolicy qp = new QueryPolicy();

RecordSet rs = client.query(qp, stmt);
while (rs.next()){
  System.out.println(rs.getKey());
  Record r = rs.getRecord();
  System.out.println(r);
}

test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1
(gen:1),(exp:453579683),(bins:(age:40))


# Cleanup

In [ ]:
%sh asadm --enable -e "manage sindex delete idx_age ns test set testset"

In [ ]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"